Toastmasters Scheduling
---

> Everyone has to do a prepared, impromptu, and evaluation speech (3 speeches on different dates) 
There are four leadership roles (president, toastmaster, table topics master, and general evaluator) --at least once but some may have two (but preferably not BOTH Presiident and Table Topics Master).

> could you make it so that a single person does NOT get BOTH President and Toastmaster roles.


The restrictions are:
- everyone has to do each of the three "main" jobs once
- the three main jobs are repeated 2 or 3 times on a single day
- students should speak "against" a different set of speakers between their two times
- auxiliary jobs should be filled with remaining students, with preference to students who haven't done that job yet

> Is "the same person shouldn't get president and toastmaster roles" over the course of the entire process, or just during a single day? over the course of the entire process.

> There should also be a "stand-in" person but not all days have a "stand-in" because on some days there will be 4 impromptu speeches or 4 prepared AND 4 evaluation speeches

> So stand-ins usually happen on days where there are no extra speakers (three speakers per day)

> 7 days

> But 23 students is a lot

In [1]:
import numpy as np
import pandas as pd

In [2]:
num_days = 7
num_students = 23

students = 'abcdefghijklmnopqrstuvwxyz'[:num_students]

full_student_list = [
        char for char in students
]

speaker_roles = ['Prepared Speaker',
                'Impromptu Speaker',
                'Evaluator'
]

prepared_speakers = ['Prepared Speaker 1',
                     'Prepared Speaker 2',
                     'Prepared Speaker 3'
]

evaluators = [
    'Evaluator 1',
    'Evaluator 2',
    'Evaluator 3'
]

impromptu_speakers = [
    'Impromptu Speaker 1',
    'Impromptu Speaker 2',
    'Impromptu Speaker 3'    
]

speakers = ['Prepared Speaker 1',
           'Impromptu Speaker 1',
           'Evaluator 1',
            'Prepared Speaker 2',
            'Impromptu Speaker 2',
            'Evaluator 2',
            'Prepared Speaker 3',
            'Impromptu Speaker 3',
            'Evaluator 3'
]

leaders = [
            'Toastmaster',
            'President',
            'Table Topics Master',
            'General Evaluator'
]

auxiliary = ['Greeter',
            'Joke Master',
            'Timer',
            'Grammarian',
            'Word of the Day',
            'Ah Counter',
            'Ballot Counter',
            'Thought of the day',
            'Sergeant at arms',
            'Stand-in'
]

Constraints:
- Everyone has to do each of the three types of speeches once and only once
- Everyone should do one of the four leadership roles once.
    - repeats should preferentially be General Evaluator
- Everyone has one and only one job per day
- Those who do not have a speech or a leadership role fill in one of the auxiliary roles.

In [3]:
#everyone has to do each of the three types of speeches once and only once

# returns True if there are no repeat assignments for the given jobs.
# people can do multiple jobs, but not not the same job twice
def check_job_repeats(calendar, jobs = []):
    #check all columns if none given
    if len(jobs) == 0:
        jobs = calendar.columns
    result = True
    for job in jobs:
        if calendar[job].duplicated().sum() > 0:
            result = False
        if not result:
            break
    return result


# nobody should have more than one job a day

# Checks if all the jobs on all the days of a dataframe are uniquely assigned --- no person has more than 1 job.
# returns True if so.
def check_day_repeats(calendar, days = []):
    if len(days) == 0:
        days = calendar.index
    result = True
    for row in days:
        if calendar.loc[row].duplicated().sum() > 0:
            result = False
        if not result:
            break
    return result

#returns true if there are no duplicates in the entire collection of job_list
def check_job_type_repeats(calendar, job_list, up_to_day):
    these_values = calendar.loc[:up_to_day, job_list].values.flatten()
    #print(these_values)
    try:
        u, c = np.unique(these_values, return_counts=True)
    except:
        print(f'Had a problem with these values:')
        print(these_values.reshape(-1,len(job_list)))
    dup = u[c > 1]
    return len(dup) == 0

Using [this stack overflow answer](https://stackoverflow.com/questions/11528078/determining-duplicate-values-in-an-array) to help with finding duplicates in the flattened numpy array.

Create a list of students to keep track of for each job. Each list 

Randomly assign students to the first set of jobs. For each job, remove that student from its acceptable list.

Randomly assign students to successive days of jobs. These could be placed together, or one at a time. Check if the student  has done the jobs before If an auxiliary job's list is empty, refill it with student names.

When using random, make sure to follow the [numpy best practice](https://numpy.org/doc/stable/reference/random/index.html) by creating an instance of a random number generator, rather than the legacy `seed()`.

In [8]:
cal = pd.DataFrame(columns = speakers + leaders + auxiliary, index = range(num_days))
cal.index.rename('day', inplace=True)


student_buckets_by_day = {
    day:full_student_list
    for day in cal.index
}
#allowed students by role
student_buckets_by_role = {
    role:full_student_list
    for role in speakers + leaders + auxiliary
}

#make an rng
seed = 2022
rng = np.random.default_rng(seed)
day_fill_counter_max = 1000
verbose = False


#for each day,
for day in cal.index:
    #say which day is being filled.
    print(f'filling schedule for day {day}.')
    #repeat the day's placement until it's right.
    day_fill_counter = 0
    day_filled = False
    while not day_filled and day_fill_counter < day_fill_counter_max:
        day_fill_counter += 1
        #refill availability bucket
        student_buckets_by_day[day] = full_student_list
        #for each role in the day,
        for role in cal.columns:
            #get the allowed students to be filled in that role
            allowed_students = np.intersect1d(student_buckets_by_day[day], student_buckets_by_role[role])
            #if that length is too short, report it and bail on filling this day.
            if len(allowed_students) == 0:
                #print(f'No more allowed students found for day {day}, role {role}.')
                break
            else:
                #pick out a student
                this_student = rng.choice(allowed_students)
                #remove the student from the day bucket
                student_buckets_by_day[day].remove(this_student)
                #assign them that role
                cal.loc[day, role] = this_student
        #after filling all the roles, check if they satisfy the rules.
        columns_ok = True
        for role in cal.columns:
            #are there any repeats for the roles?
            columns_ok = columns_ok and check_job_repeats(cal.loc[:day], [role])
            # Are there any repeats in any of the speaking and leadership roles?
            if role == prepared_speakers[-1]:
                columns_ok = columns_ok and check_job_type_repeats(cal.loc[:day], prepared_speakers, day)
            elif role == impromptu_speakers[-1]:
                columns_ok = columns_ok and check_job_type_repeats(cal.loc[:day], impromptu_speakers, day)
            elif role == evaluators[-1]:
                columns_ok = columns_ok and check_job_type_repeats(cal.loc[:day], evaluators, day)
            elif role == leaders[-1]:
                columns_ok = columns_ok and check_job_type_repeats(cal.loc[:day], leaders, day) 
        #if everything is ok, remove the students from their respective role buckets
        row_ok = check_day_repeats(cal, [day])
        if columns_ok and row_ok:
            for role in cal.columns:
                print(f'{day},{role},{cal.loc[day,role]}')
                print(f'{student_buckets_by_role[role]}')
                student_buckets_by_role[role].remove(cal.loc[day, role])
            #say we filled the day
            day_filled = True

filling schedule for day 0.
filling schedule for day 1.
filling schedule for day 2.
filling schedule for day 3.
filling schedule for day 4.
filling schedule for day 5.
filling schedule for day 6.


In [14]:
cal

,Prepared Speaker 1,Impromptu Speaker 1,Evaluator 1,Prepared Speaker 2,Impromptu Speaker 2,Evaluator 2,Prepared Speaker 3,Impromptu Speaker 3,Evaluator 3,Toastmaster,...,Greeter,Joke Master,Timer,Grammarian,Word of the Day,Ah Counter,Ballot Counter,Thought of the day,Sergeant at arms,Stand-in
day,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for col in cal.columns:
    print(f'{col}:{check_job_repeats(cal, jobs = [col])}')
    
for row in cal.index:
    print(f'{row}:{check_day_repeats(cal, [row])}')

In [11]:
row_ok

False

In [13]:
student_buckets_by_role

{'Prepared Speaker 1': [],
 'Impromptu Speaker 1': [],
 'Evaluator 1': [],
 'Prepared Speaker 2': [],
 'Impromptu Speaker 2': [],
 'Evaluator 2': [],
 'Prepared Speaker 3': [],
 'Impromptu Speaker 3': [],
 'Evaluator 3': [],
 'Toastmaster': [],
 'President': [],
 'Table Topics Master': [],
 'General Evaluator': [],
 'Greeter': [],
 'Joke Master': [],
 'Timer': [],
 'Grammarian': [],
 'Word of the Day': [],
 'Ah Counter': [],
 'Ballot Counter': [],
 'Thought of the day': [],
 'Sergeant at arms': [],
 'Stand-in': []}